In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import argparse
from prettytable import PrettyTable

In [3]:
from sampling import *

import torch.utils.data as data
from torch.utils.data import DataLoader

In [4]:
import torch
import torch.optim as optim

In [5]:
import os
import torch
import pickle
from tqdm import tqdm

In [6]:
def print_args(parse_args):
    x = PrettyTable()
    x.field_names = ["Arg.", "Value"]
    for arg in vars(parse_args):
        x.add_row([arg, getattr(parse_args, arg)])
    print(x)

In [7]:
def arg_parse():
        parser = argparse.ArgumentParser(description='LiveRec - Douyu')

        parser.add_argument('--seed', dest='seed', type=int,
            help='Random seed')

        parser.add_argument('--batch_size', dest='batch_size', type=int,
            help='Batch size - only active if torch is used')
        parser.add_argument('--seq_len', dest='seq_len', type=int,
            help='Max size of the sequence to consider')

        parser.add_argument('--num_heads', dest='num_heads', type=int,
            help='Numer of heads to use for multi-heads attention')
        parser.add_argument('--num_heads_ctx', dest='num_heads_ctx', type=int,
            help='Numer of heads to use for multi-heads attention CTX')


        parser.add_argument('--dataset', dest='dataset', 
            help='Input dataset.')
 
        parser.add_argument('--model', dest='model', type=str,
            help='Type of the model')

        parser.add_argument('--model_from', dest='mfrom', type=str,
            help='Name of the model to load')
        parser.add_argument('--model_to', dest='mto', type=str,
            help='Name of the model to save')
        parser.add_argument('--cache_dir', dest='cache_dir', type=str,
            help='Path to save the cached preprocessd dataset')
 
        parser.add_argument('--model_path', dest='model_path', type=str,
            help='Path to save the model')
        parser.add_argument('--early_stop', dest='early_stop', type=int,
            help='Number of iteration without improvment before stop')
        parser.add_argument('--ev_sample', dest='ev_sample', type=int,
            help='Number of samples for the final evaluation')
        parser.add_argument('--device', dest='device', type=str,
            help='Pytorch device')

        parser.add_argument('--lr', dest='lr', type=float,
            help='Learning rate.')
        parser.add_argument('--mask_prob', dest='mask_prob', type=float,
            help='BERT mask prob.')
        parser.add_argument('--l2', dest='l2', type=float,
            help='Strength of L2 regularization')
        parser.add_argument('--dim', dest='K', type=int,
            help='Number of latent factors')
 
        parser.add_argument('--num_iters', dest='num_iter', type=int,
            help='Number of training iterations')
        parser.add_argument('--num_epochs', dest='num_epochs', type=int,
            help='Number of training epochs')
        parser.add_argument('--num_att', dest='num_att', type=int,
            help='Num attention module for seq encoding')
        parser.add_argument('--num_att_ctx', dest='num_att_ctx', type=int,
            help='Num attention for ctx module')
 
        parser.add_argument('--topk_att', dest='topk_att', type=int,
            help='Items to send to attentive output')

        parser.add_argument('--fr_ctx', dest='fr_ctx', nargs='?',
            const=True, default=True,
            help='')
        parser.add_argument('--fr_rep', dest='fr_rep', nargs='?',
            const=True, default=False,
            help='')
        parser.add_argument('--uniform', dest='uniform', nargs='?',
            const=True, default=False,
            help='')
        parser.add_argument('--debug', dest='debug', nargs='?',
            const=True, default=False,
            help='')
#         parser.add_argument('--caching', dest='caching', nargs='?',
#             const=True, default=False,
#             help='')
        parser.add_argument('--caching', dest='caching', nargs='?',
            const=True, default=True,
            help='')

        parser.set_defaults(
                        seed=42,
                        dataset="",
                        lr=0.0005,
                        l2=0.1,  
                        mask_prob=0.5,  
                        batch_size=100, 
                        num_att=2,
                        num_att_ctx=2,
                        num_heads=4,
                        num_heads_ctx=4,
                        num_iter=200,
                        seq_len=16,  
                        topk_att=64,  
                        early_stop=15,  
                        K=64,  
                        num_epochs=150,
#                         model="LiveRec",
                        model="POP",
                        model_path="",
                        mto="liverec",
                        device="cuda",
                        cache_dir=""
                       )


        args, unknown = parser.parse_known_args()
        return args 

In [8]:
args = arg_parse()
print_args(args)
args.device = torch.device(args.device)

+---------------+---------+
|      Arg.     |  Value  |
+---------------+---------+
|      seed     |    42   |
|   batch_size  |   100   |
|    seq_len    |    16   |
|   num_heads   |    4    |
| num_heads_ctx |    4    |
|    dataset    |         |
|     model     |   POP   |
|     mfrom     |   None  |
|      mto      | liverec |
|   cache_dir   |         |
|   model_path  |         |
|   early_stop  |    15   |
|   ev_sample   |   None  |
|     device    |   cuda  |
|       lr      |  0.0005 |
|   mask_prob   |   0.5   |
|       l2      |   0.1   |
|       K       |    64   |
|    num_iter   |   200   |
|   num_epochs  |   150   |
|    num_att    |    2    |
|  num_att_ctx  |    2    |
|    topk_att   |    64   |
|     fr_ctx    |   True  |
|     fr_rep    |  False  |
|    uniform    |  False  |
|     debug     |  False  |
|    caching    |   True  |
+---------------+---------+


In [9]:
INFILE = os.path.join(args.dataset,'douyu_10k_min_duration.csv')
#user,streamer_id,start,stop
cols = ["user","streamer","start","stop","duration"]
data_fu = pd.read_csv(INFILE, header=None, names=cols)
    
# Add one for padding
data_fu.user = pd.factorize(data_fu.user)[0]+1
data_fu['streamer_raw'] = data_fu.streamer
data_fu.streamer = pd.factorize(data_fu.streamer)[0]+1
print("Num users: ", data_fu.user.nunique())
print("Num streamers: ", data_fu.streamer.nunique())
print("Num interactions: ", len(data_fu))

Num users:  11610
Num streamers:  47612
Num interactions:  5378125


In [10]:
data_fu.start.min(),data_fu.start.max(),data_fu.stop.min(),data_fu.stop.max()

(0, 87839, 1, 87840)

In [11]:
args.M = data_fu.user.max()+1 # users
args.N = data_fu.streamer.max()+2 # items
args.D = data_fu.duration.max()+1 # duration

data_temp = data_fu.drop_duplicates(subset=['streamer','streamer_raw'])
umap      = dict(zip(data_temp.streamer_raw.tolist(),data_temp.streamer.tolist()))

In [12]:
args.M ,args.N ,args.D

(11611, 47614, 2883)

In [13]:
# Splitting and caching
max_step = max(data_fu.start.max(),data_fu.stop.max())
print("Num timesteps: ", max_step)
args.max_step = max_step
args.pivot_1  = max_step-500
args.pivot_2  = max_step-250

Num timesteps:  87840


In [14]:
print("caching availability")
ts = {}
max_avail = 0
for s in range(max_step):
    all_av = data_fu[(data_fu.start<=s) & (data_fu.stop>s)].streamer.unique().tolist()
    ts[s] = all_av
    max_avail = max(max_avail,len(ts[s]))
args.max_avail = max_avail
args.ts = ts
print("max_avail: ", max_avail)

caching availability
max_avail:  446


In [15]:
# Compute availability matrix of size (num_timesteps x max_available)
max_av   = max([len(v) for k,v in args.ts.items()])
max_step = max([k for k,v in args.ts.items()])+1
av_tens = torch.zeros(max_step,max_av).type(torch.long)
for k,v in args.ts.items():
    av_tens[k,:len(v)] = torch.LongTensor(v)
args.av_tens = av_tens.to(args.device)

In [16]:
data_fu

,user,streamer,start,stop,duration,streamer_raw
0,1,1,81824,81825,2,1211089da67
1,1,2,73720,73721,1,eb5bf69ab60
2,1,1,81825,81834,18,1211089da67
3,1,2,73673,73676,4,eb5bf69ab60
4,1,1,81745,81750,9,1211089da67
...,...,...,...,...,...,...
5378120,11610,8849,21366,21368,2,80ec36cddc2
5378121,11610,8849,21364,21367,4,80ec36cddc2
5378122,11610,8849,21319,21332,24,80ec36cddc2
5378123,11610,8849,21362,21365,4,80ec36cddc2


In [17]:
def custom_collate(batch,args):
    # returns a [batch x seq x feats] tensor
    # feats: [padded_positions,positions,inputs_ts,items,users,targets,targets_ts]

    bs = len(batch)
    feat_len = len(batch[0])
    batch_seq = torch.zeros(bs,args.seq_len, feat_len, dtype=torch.long)
    for ib,b in enumerate(batch):
        for ifeat,feat in enumerate(b):
            batch_seq[ib,b[0],ifeat] = feat
    return batch_seq

In [18]:
class SequenceDataset(data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    lengths = torch.tensor([ t.shape[0] for t in batch ]).to(device)
    ## padd
    batch = [ torch.Tensor(t).to(device) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    mask = (batch != 0).to(device)
    return batch, lengths, mask

def get_sequences(_data, _p1, _p2, args, max_u=int(10e9)):
    data_list = []

    _data = _data[_data.stop<_p2].copy()
    
    grouped = _data.groupby('user')
    for user_id, group in tqdm(grouped):
        group = group.sort_values('start')
        group = group.tail(args.seq_len+1)
        if len(group)<2: continue

        group = group.reset_index(drop=True) 
        
        # Get last interaction
        last_el = group.tail(1)
        yt = last_el.start.values[0]
        group.drop(last_el.index,inplace=True)

        # avoid including train in test/validation
        if yt < _p1 or yt >= _p2: continue

        padlen = args.seq_len - len(group)

        # sequence input features
        positions  = torch.LongTensor(group.index.values)
        inputs_ts  = torch.LongTensor(group.start.values)
        items      = torch.LongTensor(group['streamer'].values)
        duration   = torch.LongTensor(group['duration'].values)
        users      = torch.LongTensor(group.user.values)
        bpad       = torch.LongTensor(group.index.values + padlen)
        diff_du    = torch.LongTensor(np.diff(np.array(inputs_ts[1:].tolist() + [last_el.start.values[0]])))

        # sequence output features
        targets    = torch.LongTensor(items[1:].tolist() + [last_el.streamer.values[0]])
        targets_ts = torch.LongTensor(inputs_ts[1:].tolist() + [last_el.start.values[0]])

        data_list.append([bpad,positions,inputs_ts,items,users,targets,targets_ts,duration])

        # stop if user limit is reached
        if len(data_list)>max_u: break

    return SequenceDataset(data_list)

In [19]:
_data = data_fu[data_fu.stop<50000].copy()
grouped = _data.groupby('user')
group = grouped.get_group(1)
group

,user,streamer,start,stop,duration,streamer_raw
6,1,4,39495,39496,1,0f7563e45a4
11,1,8,15434,15435,1,dd309e40138
12,1,9,16869,16871,2,4219c23771f
13,1,10,15441,15445,6,cf9ddf104ef
21,1,16,18527,18529,2,c7d5eaa65e3
...,...,...,...,...,...,...
458,1,18,15486,15492,10,cdd268ecabe
460,1,18,15444,15460,30,cdd268ecabe
469,1,18,17841,17846,8,cdd268ecabe
471,1,18,18543,18552,16,cdd268ecabe


In [20]:
group = group.sort_values('start')
group = group.tail(args.seq_len+1)
group = group.reset_index(drop=True)

In [21]:
last_el = group.tail(1)
yt = last_el.start.values[0]
group.drop(last_el.index,inplace=True)
group

,user,streamer,start,stop,duration,streamer_raw
0,1,74,39485,39486,2,c0a19ec96d5
1,1,70,39486,39490,6,ae93a532e9b
2,1,84,39486,39487,1,994cd41f6bd
3,1,18,39492,39493,1,cdd268ecabe
4,1,18,39492,39496,6,cdd268ecabe
5,1,114,39492,39493,1,0ef17bcf67e
6,1,18,39495,39496,1,cdd268ecabe
7,1,4,39495,39496,1,0f7563e45a4
8,1,25,39513,39514,1,befb02c9c8f
9,1,110,39513,39514,1,830661aa583


In [22]:
padlen = args.seq_len - len(group)

bpad       = torch.LongTensor(group.index.values + padlen)
positions  = torch.LongTensor(group.index.values)
inputs_ts  = torch.LongTensor(group.start.values)
items      = torch.LongTensor(group['streamer'].values)
duration   = torch.LongTensor(group['duration'].values)
users      = torch.LongTensor(group.user.values)
targets    = torch.LongTensor(items[1:].tolist() + [last_el.streamer.values[0]])
targets_ts = torch.LongTensor(inputs_ts[1:].tolist() + [last_el.start.values[0]])
diff_du    = torch.LongTensor(np.diff(np.array(inputs_ts[1:].tolist() + [last_el.start.values[0]])))

bpad,positions,inputs_ts,items,users,targets,targets_ts,duration,diff_du

(tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
 tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
 tensor([39485, 39486, 39486, 39492, 39492, 39492, 39495, 39495, 39513, 39513,
         39514, 39515, 42801, 42813, 42814, 42814]),
 tensor([ 74,  70,  84,  18,  18, 114,  18,   4,  25, 110,  24,  90,  78,  18,
         118, 118]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 tensor([ 70,  84,  18,  18, 114,  18,   4,  25, 110,  24,  90,  78,  18, 118,
         118,  56]),
 tensor([39486, 39486, 39492, 39492, 39492, 39495, 39495, 39513, 39513, 39514,
         39515, 42801, 42813, 42814, 42814, 42816]),
 tensor([ 2,  6,  1,  1,  6,  1,  1,  1,  1,  1,  2,  1, 24,  1,  3,  2]),
 tensor([   0,    6,    0,    0,    3,    0,   18,    0,    1,    1, 3286,   12,
            1,    0,    2]))

In [29]:
data_list = []
data_list.append([bpad,positions,inputs_ts,items,users,targets,targets_ts,duration])
data_list

[[tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
  tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
  tensor([39485, 39486, 39486, 39492, 39492, 39492, 39495, 39495, 39513, 39513,
          39514, 39515, 42801, 42813, 42814, 42814]),
  tensor([ 74,  70,  84,  18,  18, 114,  18,   4,  25, 110,  24,  90,  78,  18,
          118, 118]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  tensor([ 70,  84,  18,  18, 114,  18,   4,  25, 110,  24,  90,  78,  18, 118,
          118,  56]),
  tensor([39486, 39486, 39492, 39492, 39492, 39495, 39495, 39513, 39513, 39514,
          39515, 42801, 42813, 42814, 42814, 42816]),
  tensor([ 2,  6,  1,  1,  6,  1,  1,  1,  1,  1,  2,  1, 24,  1,  3,  2])]]

In [21]:
cache_tr = os.path.join(args.cache_dir,"douyu_10k_min_dur_tr.p")
cache_te = os.path.join(args.cache_dir,"douyu_10k_min_dur_te.p")
cache_va = os.path.join(args.cache_dir,"douyu_10k_min_dur_val.p")

mu = int(10e9)

In [22]:
datalist_tr = get_sequences(data_fu,0,args.pivot_1,args,mu)
datalist_va = get_sequences(data_fu,args.pivot_1,args.pivot_2,args,mu)
datalist_te = get_sequences(data_fu,args.pivot_2,max_step,args,mu)
    
pickle.dump(datalist_te, open(cache_te, "wb"))
pickle.dump(datalist_tr, open(cache_tr, "wb"))
pickle.dump(datalist_va, open(cache_va, "wb"))

100%|██████████████████████████████████████████████████████████████████████████| 11610/11610 [00:04<00:00, 2385.91it/s]


In [23]:
train_loader = DataLoader(datalist_tr,batch_size=args.batch_size,
                              collate_fn=lambda x: custom_collate(x,args))
val_loader   = DataLoader(datalist_va,batch_size=args.batch_size,
                              collate_fn=lambda x: custom_collate(x,args))
test_loader  = DataLoader(datalist_te,batch_size=args.batch_size,
                              collate_fn=lambda x: custom_collate(x,args))

In [24]:
from eval import *
from data import *
from models import *

In [25]:
args.model = 'POP'
MPATH1,MODEL1 = get_model_type(args)
data_tr = data_fu[data_fu.stop<args.pivot_1]
model1 = MODEL1(args,data_tr)

MPATH1,MODEL1

('liverec_64_0.1_64_2_16_ctx.pt', baselines.POP)

In [26]:
scores = compute_recall(model1, test_loader, args)
print("Final score")
print("="*11)
print_scores(scores)

24it [00:02,  8.77it/s]

Final score
all: h@1: 0.01857 h@5: 0.18700 h@10: 0.28113 ndcg@1: 0.01857 ndcg@5: 0.10033 ndcg@10: 0.13027
new: h@1: 0.02203 h@5: 0.13341 h@10: 0.19829 ndcg@1: 0.02203 ndcg@5: 0.07873 ndcg@10: 0.09917
rep: h@1: 0.01675 h@5: 0.21521 h@10: 0.32474 ndcg@1: 0.01675 ndcg@5: 0.11170 ndcg@10: 0.14664
ratio:  0.6551287463064585


In [27]:
args.model = 'REP'
MPATH2,MODEL2 = get_model_type(args)
data_tr = data_fu[data_fu.stop<args.pivot_1]
model2 = MODEL2(args,data_tr)

MPATH2,MODEL2

('liverec_64_0.1_64_2_16_ctx.pt', baselines.REP)

In [28]:
scores = compute_recall(model2, test_loader, args)
print("Final score")
print("="*11)
print_scores(scores)

24it [00:02, 10.01it/s]

Final score
all: h@1: 0.33727 h@5: 0.59688 h@10: 0.62263 ndcg@1: 0.33727 ndcg@5: 0.47827 ndcg@10: 0.48697
new: h@1: 0.00245 h@5: 0.01958 h@10: 0.03672 ndcg@1: 0.00245 ndcg@5: 0.01090 ndcg@10: 0.01628
rep: h@1: 0.51353 h@5: 0.90077 h@10: 0.93106 ndcg@1: 0.51353 ndcg@5: 0.72430 ndcg@10: 0.73476
ratio:  0.6551287463064585


In [29]:
args.model='MF'
MPATH,MODEL3 = get_model_type(args)
MPATH,MODEL3
model3 = MODEL3(args).to(args.device)
optimizer = optim.Adam(model3.parameters(),lr=args.lr,weight_decay=args.l2)

best_val = 0.0
best_max = args.early_stop
best_cnt = best_max

In [30]:
print("training...")
for epoch in range(args.num_epochs):
    loss_all = 0.0; loss_cnt = 0
    model3.train()
    for data in tqdm(train_loader):
        data = data.to(args.device)
        optimizer.zero_grad()
    
        loss = model3.train_step(data)
            
        loss_all += loss.item()
        loss_cnt += (data[:,:,5]!=0).sum()
            
        loss.backward()
        optimizer.step()
            
        if torch.isnan(loss):
            print("loss is nan !") 
    
    scores = compute_recall(model3, val_loader, args, maxit=500)
    print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss_all/loss_cnt))
    print_scores(scores)
    
    hall = scores['all']['h01']
    if hall>best_val:
        best_val = hall
        torch.save(model3.state_dict(), MPATH)
        best_cnt = best_max
    else:
        best_cnt -= 1
        if best_cnt == 0:
            break
    
model3 = MODEL3(args).to(args.device)
model3.load_state_dict(torch.load(MPATH))
    
scores = compute_recall(model3, test_loader, args)
print("Final score")
print("="*11)
print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss_all/loss_cnt))
print_scores(scores)
save_scores(scores,args)

training...


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:28<00:00,  4.06it/s]
17it [00:01,  9.02it/s]


Epoch: 000, Loss: 4.56707
all: h@1: 0.00549 h@5: 0.03112 h@10: 0.05369 ndcg@1: 0.00549 ndcg@5: 0.01772 ndcg@10: 0.02509
new: h@1: 0.00448 h@5: 0.02392 h@10: 0.04634 ndcg@1: 0.00448 ndcg@5: 0.01410 ndcg@10: 0.02131
rep: h@1: 0.00619 h@5: 0.03608 h@10: 0.05876 ndcg@1: 0.00619 ndcg@5: 0.02022 ndcg@10: 0.02770
ratio:  0.5918242830994509


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:28<00:00,  4.01it/s]
17it [00:01, 11.39it/s]


Epoch: 001, Loss: 4.38042
all: h@1: 0.00488 h@5: 0.03295 h@10: 0.05430 ndcg@1: 0.00488 ndcg@5: 0.01824 ndcg@10: 0.02519
new: h@1: 0.00299 h@5: 0.02541 h@10: 0.04634 ndcg@1: 0.00299 ndcg@5: 0.01426 ndcg@10: 0.02096
rep: h@1: 0.00619 h@5: 0.03814 h@10: 0.05979 ndcg@1: 0.00619 ndcg@5: 0.02099 ndcg@10: 0.02811
ratio:  0.5918242830994509


...

Epoch: 117, Loss: 0.42802
all: h@1: 0.07993 h@5: 0.21538 h@10: 0.31788 ndcg@1: 0.07993 ndcg@5: 0.14794 ndcg@10: 0.18093
new: h@1: 0.03139 h@5: 0.13453 h@10: 0.20478 ndcg@1: 0.03139 ndcg@5: 0.08156 ndcg@10: 0.10421
rep: h@1: 0.11340 h@5: 0.27113 h@10: 0.39588 ndcg@1: 0.11340 ndcg@5: 0.19372 ndcg@10: 0.23384
ratio:  0.5918242830994509


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:30<00:00,  3.70it/s]
17it [00:01,  8.87it/s]


Epoch: 118, Loss: 0.42797
all: h@1: 0.08054 h@5: 0.21599 h@10: 0.31971 ndcg@1: 0.08054 ndcg@5: 0.14819 ndcg@10: 0.18151
new: h@1: 0.02990 h@5: 0.13453 h@10: 0.20777 ndcg@1: 0.02990 ndcg@5: 0.08140 ndcg@10: 0.10487
rep: h@1: 0.11546 h@5: 0.27216 h@10: 0.39691 ndcg@1: 0.11546 ndcg@5: 0.19426 ndcg@10: 0.23436
ratio:  0.5918242830994509


24it [00:02,  8.84it/s]

Final score
Epoch: 118, Loss: 0.42797
all: h@1: 0.08485 h@5: 0.21824 h@10: 0.31406 ndcg@1: 0.08485 ndcg@5: 0.15253 ndcg@10: 0.18340
new: h@1: 0.03060 h@5: 0.12607 h@10: 0.19584 ndcg@1: 0.03060 ndcg@5: 0.08027 ndcg@10: 0.10280
rep: h@1: 0.11340 h@5: 0.26675 h@10: 0.37629 ndcg@1: 0.11340 ndcg@5: 0.19057 ndcg@10: 0.22583
ratio:  0.6551287463064585


In [31]:
args.model='FPMC'
MPATH,MODEL4 = get_model_type(args)
model4 = MODEL4(args).to(args.device)
optimizer = optim.Adam(model4.parameters(),lr=args.lr,weight_decay=args.l2)
best_val = 0.0
best_max = args.early_stop
best_cnt = best_max

In [32]:
print("training...")
for epoch in range(args.num_epochs):
    loss_all = 0.0; loss_cnt = 0
    model4.train()
    for data in tqdm(train_loader):
        data = data.to(args.device)
        optimizer.zero_grad()
    
        loss = model4.train_step(data)
            
        loss_all += loss.item()
        loss_cnt += (data[:,:,5]!=0).sum()
            
        loss.backward()
        optimizer.step()
            
        if torch.isnan(loss):
            print("loss is nan !") 
    
    scores = compute_recall(model4, val_loader, args, maxit=500)
    print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss_all/loss_cnt))
    print_scores(scores)
    
    hall = scores['all']['h01']
    if hall>best_val:
        best_val = hall
        torch.save(model4.state_dict(), MPATH)
        best_cnt = best_max
    else:
        best_cnt -= 1
        if best_cnt == 0:
            break
    
model4 = MODEL4(args).to(args.device)
model4.load_state_dict(torch.load(MPATH))
    
scores = compute_recall(model4, test_loader, args)
print("Final score")
print("="*11)
print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss_all/loss_cnt))
print_scores(scores)
save_scores(scores,args)

training...


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:25<00:00,  4.49it/s]
17it [00:05,  2.92it/s]


Epoch: 000, Loss: 6.06308
all: h@1: 0.00671 h@5: 0.02929 h@10: 0.06040 ndcg@1: 0.00671 ndcg@5: 0.01850 ndcg@10: 0.02834
new: h@1: 0.00598 h@5: 0.02392 h@10: 0.05082 ndcg@1: 0.00598 ndcg@5: 0.01545 ndcg@10: 0.02399
rep: h@1: 0.00722 h@5: 0.03299 h@10: 0.06701 ndcg@1: 0.00722 ndcg@5: 0.02060 ndcg@10: 0.03133
ratio:  0.5918242830994509


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:28<00:00,  4.00it/s]
17it [00:05,  3.32it/s]


Epoch: 001, Loss: 5.61099
all: h@1: 0.00732 h@5: 0.03295 h@10: 0.06833 ndcg@1: 0.00732 ndcg@5: 0.02049 ndcg@10: 0.03159
new: h@1: 0.00598 h@5: 0.02392 h@10: 0.05232 ndcg@1: 0.00598 ndcg@5: 0.01545 ndcg@10: 0.02446
rep: h@1: 0.00825 h@5: 0.03918 h@10: 0.07938 ndcg@1: 0.00825 ndcg@5: 0.02397 ndcg@10: 0.03651
ratio:  0.5918242830994509


...

Epoch: 116, Loss: 0.35662
all: h@1: 0.18182 h@5: 0.35265 h@10: 0.44539 ndcg@1: 0.18182 ndcg@5: 0.27098 ndcg@10: 0.30088
new: h@1: 0.02691 h@5: 0.15097 h@10: 0.24813 ndcg@1: 0.02691 ndcg@5: 0.08912 ndcg@10: 0.12028
rep: h@1: 0.28866 h@5: 0.49175 h@10: 0.58144 ndcg@1: 0.28866 ndcg@5: 0.39641 ndcg@10: 0.42543
ratio:  0.5918242830994509


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:26<00:00,  4.29it/s]
17it [00:05,  2.99it/s]


Epoch: 117, Loss: 0.35684
all: h@1: 0.17816 h@5: 0.35326 h@10: 0.44600 ndcg@1: 0.17816 ndcg@5: 0.26977 ndcg@10: 0.29967
new: h@1: 0.02990 h@5: 0.15247 h@10: 0.24963 ndcg@1: 0.02990 ndcg@5: 0.09067 ndcg@10: 0.12178
rep: h@1: 0.28041 h@5: 0.49175 h@10: 0.58144 ndcg@1: 0.28041 ndcg@5: 0.39329 ndcg@10: 0.42236
ratio:  0.5918242830994509


24it [00:09,  2.58it/s]

Final score
Epoch: 117, Loss: 0.35684
all: h@1: 0.18024 h@5: 0.33727 h@10: 0.41959 ndcg@1: 0.18024 ndcg@5: 0.26280 ndcg@10: 0.28948
new: h@1: 0.02203 h@5: 0.13341 h@10: 0.22521 ndcg@1: 0.02203 ndcg@5: 0.08079 ndcg@10: 0.11037
rep: h@1: 0.26353 h@5: 0.44459 h@10: 0.52191 ndcg@1: 0.26353 ndcg@5: 0.35861 ndcg@10: 0.38377
ratio:  0.6551287463064585


In [ ]:
class GRUModel(nn.Module):
    def __init__(self, args,data_tr):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
        self.item_bias = torch.nn.Embedding(args.N+1, 1, padding_idx=0)
        self.item_embedding = torch.nn.Embedding(args.N+1, args.K, padding_idx=0)
        self.user_embedding = torch.nn.Embedding(args.M+1, args.K, padding_idx=0)
        self.dur_emb = nn.Embedding(args.D+1, args.K, padding_idx=0)
        
    def forward(self, x):
        # x shape: (batch_size, seq_length, input_size)
        # hidden shape: (num_layers * num_directions, batch_size, hidden_size)
        out, hidden = self.gru(x)
        # out shape: (batch_size, seq_length, hidden_size * num_directions)
        # hidden shape: (num_layers * num_directions, batch_size, hidden_size)
        out = out[:, -1, :] # get the last output of the sequence
        # out shape: (batch_size, hidden_size)
        out = self.fc(out) # apply a linear layer
        # out shape: (batch_size, output_size)
        return out
        

    
    
    def forward(self,users,items):
        ui = self.user_embedding(users)
        ii = self.item_embedding(items)
        di = self.item_embedding(items)
        ib = self.item_bias(items).squeeze()
        return (ui * ii).sum(-1) + ib

    def train_step(self, data, use_ctx=False): # for training
        bs     = data.shape[0]
        inputs = data[:,:,3] 
        pos    = data[:,:,5] 
        users  = data[:,:,4] 
        neg    = sample_negs(data,self.args).to(self.args.device)
        
        inputs = data[:,:,3] # inputs 
        pos   = data[:,:,5] # targets
        xtsy  = data[:,:,6] # targets ts
        dur   = data[:,:,7] # duration

        pos_logits = self(users,pos)
        neg_logits = self(users,neg) 

        loss = - (pos_logits - neg_logits).sigmoid().log()
        loss = loss[inputs!=0].sum()

        return loss

    def compute_rank(self,data,store,k=10):
        inputs = data[:,:,3] 
        pos    = data[:,:,5] 
        users  = data[:,:,4] 
        xtsy   = data[:,:,6] # targets ts

        mask = torch.ones_like(pos[:,-1]).type(torch.bool) # (1,1,1,1,1,1,1)
        for b in range(pos.shape[0]):
            avt = pos[b,:-1]
            avt = avt[avt!=0] # avt: targets
            mask[b] = pos[b,-1] in avt
            store['ratio'] += [float(pos[b,-1] in avt)]
            
        for b in range(inputs.shape[0]):
            step = xtsy[b,-1].item()
            av = torch.LongTensor(self.args.ts[step]).to(self.args.device)
            av_embs = self.item_embedding(av)
 
            if self.mf_avg:
                inp = inputs[b,:]
                mean_vec = self.item_embedding(inp[inp!=0])
                scores  = (mean_vec.mean(0).unsqueeze(0) * av_embs).sum(-1) 
                scores += self.item_bias(av).squeeze()
            else:
                u_vec = self.user_embedding(users[b,-1])
                scores  = (u_vec.unsqueeze(0) * av_embs).sum(-1) 
                scores += self.item_bias(av).squeeze()

            iseq = pos[b,-1] == av
            idx = torch.where(iseq)
            idx = idx[0] 
            rank = torch.where(torch.argsort(scores, descending=True)==idx)[0].item()

            if mask[b]: # rep
                store['rrep'] += [rank]
            else:
                store['rnew'] += [rank]
            store['rall'] += [rank]
        return store